In [1]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
from collections import Counter
import os
from ckiptagger import WS, POS, NER
import datetime

In [2]:
bsn_links =['60030', '60001', '60559']
bsn_categories=['電腦應用綜合討論', '電視遊樂器綜合討論', '智慧型手機']
base_url = 'https://forum.gamer.com.tw/'

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
}

In [3]:
def get_article_url_list(forum_url):
    num = 2
    if (classify == 1.0):
        num = 21 #爬20頁
    elif (classify == 2.0):
        num = 5
    article_url_list = []
    
    for pageNum in range(1,num):  # 爬 1 頁內容
        print(pageNum)
        target_url = forum_url + "&page=" + str(pageNum)
        r = requests.get(target_url, headers=HEADERS)
        if r.status_code != requests.codes.ok:
            print("載入失敗")
            return []
    
        soup = BeautifulSoup(r.text,'lxml')
        item_blocks = soup.select('table.b-list > tr[class="b-list__row b-list-item b-imglist-item"]')
        for item_block in item_blocks:
            title_block = item_block.select_one('.b-list__main__title')
            article_url = f"https://forum.gamer.com.tw/{title_block.get('href')}"
            article_url_list.append(article_url)
        time.sleep(2)
        
    return article_url_list


#主題總頁數
def get_article_total_page(soup):
    article_total_page = soup.select_one('.BH-pagebtnA > a:last-of-type').text
    return int(article_total_page)




#主題頁面資訊
def get_article_info(article_url, count):
    
    soup = BeautifulSoup(requests.get(article_url, headers=HEADERS).text,'lxml')


    article_title = soup.select_one('h1.c-post__header__title').text
    
    article_total_page = get_article_total_page(soup)  #獲得總樓層的數量
    
    reply_info_list = []
    for page in range(article_total_page):
        if (count < 10000):
            crawler_url = f"{article_url}&page={page + 1}"
            count = get_reply_info_list(crawler_url, count) #爬頁面留言
            random.uniform(1, 3)
    return count
    
    
    
    
def get_reply_info_list(url, count):
    soup = BeautifulSoup(requests.get(url, headers=HEADERS).text,'lxml')
    reply_blocks = soup.select('section[id^="post_"]')


    for reply_block in reply_blocks:
        #找主區塊評論
        reply_info = reply_block.select_one('.c-article__content').text
        reply_info = re.sub(r'\n+',"", reply_info)
        reply_info = re.sub(r'\※ 引述《.*\》之銘言',"", reply_info)

        #找主區塊日期
        date_info = reply_block.select_one('div[class="c-post__header__info"] > a[class="edittime tippy-post-info"]').get('data-mtime')
        date_info = re.split(r" ",date_info)
        #print(date_info[0])


        contents.append(reply_info)
        commentDate.append(date_info[0])
        categories.append(category)
        classifies.append(classify)
        count += 1

        #選擇該回復下的小回復區塊
        reply_comment_info = reply_block.select_one('div[id^="Commendlist_"]')
        #擷取每一個小回復區塊
        for the_comment_block in reply_comment_info.select('div[class="reply-content"]'):
            reply_info = the_comment_block.select_one('article[class="reply-content__article c-article"] > span').text
            reply_info = re.sub(r'\[.*\]',"",reply_info)


            date_info = the_comment_block.select_one('div[class="edittime"]').get('title')
            date_info = re.split(r" ",date_info)
            #print(date_info[1])
            #print(reply_info)

            contents.append(reply_info)
            commentDate.append(date_info[1])
            categories.append(category)
            classifies.append(classify)
            count += 1
    return count


def word_frequency( wp_pair ):
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
    return 0
    

In [4]:
%%time
contents = []
categories = []
classifies = []
commentDate = []



for i, url_short_name in enumerate(bsn_links):  #抓類別專版
    if (url_short_name == '60030'):
        category = "電腦應用綜合討論"
        classify = 0.0
        count = 0
    elif (url_short_name == '60001'):
        category = "電視遊樂器綜合討論"
        classify = 1.0
        count = 0
    elif (url_short_name == '60559'):
        category = "智慧型手機"
        classify = 2.0
        count = 0

    
    B_url = base_url + 'B.php?bsn=' + url_short_name

    article_url_list = get_article_url_list(B_url) #獲得該頁所有主題URL資訊
    
    for article_url in article_url_list:
        if (count < 10000):
            count = get_article_info(article_url, count) #獲取頁面資訊

1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
Wall time: 5min 32s


In [5]:
data = zip(categories, contents, commentDate, classifies)
df = pd.DataFrame(list(data), columns=['category' ,'content','date', 'classify'])
df.tail(5)

,category,content,date,classify
29399,智慧型手機,想請問 小米 MAX3MIUI版本 MIUI Global 11.0.1 | 穩定版 目前有...,2020-03-29,2.0
29400,智慧型手機,請教一下小米之家一條95元的編織線有支援快充嗎,2020-03-29,2.0
29401,智慧型手機,看你是多快 基本上18w一定沒問題,2020-03-29,2.0
29402,智慧型手機,感恩 18w就夠了,2020-03-29,2.0
29403,智慧型手機,只要不是買那種夜市或是淘寶太過便宜的線 18w都沒問題的,2020-03-29,2.0


In [6]:
df.groupby('category').size()

category
智慧型手機        10003
電腦應用綜合討論     10071
電視遊樂器綜合討論     9330
dtype: int64

# 若model無法動，將 model_path改成絕對路徑

In [7]:
wd = os.getcwd()

model_path = wd + '\ckiplab-model-data'

ws = WS(model_path)
pos = POS(model_path)
ner = NER(model_path)

In [8]:
category_freq = []
word = []


pattern = re.compile(r'[\u4e00-\u9fa5]+')

tokens = ws(df.content)


In [9]:
tokens_list = []
tokens_str = []
drops = []

for idx,line in enumerate(tokens):
    # 刪除不屬於中文字元、標點符號、特殊符號、英文、與數字
    line_list = [x for x in line if pattern.match(x)]
    line_str = " ".join(line_list) # 合併成字串 以空格分隔
    # 過濾之後，有些留言可能會變成空值 需要進一步刪除該筆
    if len(line_str)==0:
        #print(idx, ':skip a blank line跳開這筆空白內容')
        drops.append(idx)
        continue
    tokens_list.append(line_list)
    tokens_str.append(line_str)

In [10]:
tokens_pos = pos(tokens_list)

word_pos_pair = [list(zip(w,p)) for w, p in zip(tokens_list, tokens_pos)]

with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
    stops = f.read().split('\n') 

allowPOS=['Na','Nb','Nc','VA','VAC','VB','VC']

tokens_v2 =[]
for wp in word_pos_pair:
    tokens_v2.append([w for w,p in wp if w not in stops and (len(w) >= 2) and p in allowPOS])

tokens_pos = pos(tokens_v2)
word_pos_pair = [list(zip(w,p)) for w, p in zip(tokens_v2, tokens_pos)]

In [11]:
entity_list = ner(tokens_v2, tokens_pos)

In [12]:
category_freq = []
for wp in word_pos_pair:
    keyfreqs =[]
    filtered_words =[]
    word_frequency(wp)
    counter = Counter(filtered_words)
    keyfreqs.append(counter.most_common(200))
    category_freq.append(keyfreqs[0])

In [13]:
#print(tokens_v2[0])
#print(tokens_pos[8])
#print(entity_list[0])
#print(category_freq[3])
#print(keyfreqs[0])

In [14]:
#df['tokens'] = word
#df['freq'] = category_freq

data = zip(commentDate, categories, contents, classifies, tokens_list, tokens_str, category_freq, entity_list, word_pos_pair)
df2 = pd.DataFrame(list(data), columns=['date', 'category', 'content', 'classify', 'tokens', 'tokens_str', 'freq', 'entities', 'token_pos'])


df2.head(5)

,date,category,content,classify,tokens,tokens_str,freq,entities,token_pos
0,2021-06-17,電腦應用綜合討論,消息來源：https://www.ithome.com/0/557/907.htmIT之家6...,0.0,"[消息, 來源, 之, 家, 月, 消息, 近日, 微軟, 即將, 發布, 的, 全新, 操...",消息 來源 之 家 月 消息 近日 微軟 即將 發布 的 全新 操作 系統 遭到 洩露 安裝...,"[(系統, 7), (微軟, 4), (洩露, 4), (版本, 4), (消息, 2), ...","{(6, 8, ORG, 微軟), (37, 39, ORG, 微軟), (87, 89, ...","[(消息, Na), (來源, Na), (消息, Na), (微軟, Nb), (發布, ..."
1,2021-06-18,電腦應用綜合討論,我覺得提前洩漏基本上已經是一種營銷手段了…但還是好香,0.0,"[我, 覺得, 提前, 洩漏, 基本, 上, 已經, 是, 一, 種, 營銷, 手段, 了,...",我 覺得 提前 洩漏 基本 上 已經 是 一 種 營銷 手段 了 但 還是 好 香,"[(提前, 1), (洩漏, 1), (營銷, 1), (手段, 1)]",{},"[(提前, VB), (洩漏, VC), (營銷, Na), (手段, Na)]"
2,2021-06-18,電腦應用綜合討論,商業操作。,0.0,"[商業, 操作]",商業 操作,"[(商業, 1), (操作, 1)]",{},"[(商業, Na), (操作, VC)]"
3,2021-06-18,電腦應用綜合討論,"Alpha test phase I, 開放體驗板下載. 免費測試員招募中.",0.0,"[開放, 體驗板, 下載, 免費, 測試員, 招募, 中]",開放 體驗板 下載 免費 測試員 招募 中,"[(開放, 1), (體驗板, 1), (下載, 1), (測試員, 1), (招募, 1)]",{},"[(開放, VC), (體驗板, Na), (下載, VC), (測試員, Na), (招募..."
4,2021-06-18,電腦應用綜合討論,"按照慣例,發行後過約一年才適合一般消費者使用",0.0,"[按照, 慣例, 發行, 後, 過, 約, 一, 年, 才, 適合, 一般, 消費者, 使用]",按照 慣例 發行 後 過 約 一 年 才 適合 一般 消費者 使用,"[(慣例, 1), (發行, 1), (消費者, 1), (使用, 1)]",{},"[(慣例, Na), (發行, VC), (消費者, Na), (使用, VC)]"


In [15]:
df2.shape[0]

28409

In [16]:
df2.to_csv('./dataset/bahamut_preprocessed_data.csv', sep='|', index=None)

In [17]:
#computer_content = [] #電腦版資料
#console_content = [] #家機版資料
#phone_content = [] #手機板資料

#for i in range(0, len(df2)):
    
    #if(df.category[i] == "電腦應用綜合討論"):
        #computer_content.append(df2.tokens[i])
    #elif(df.category[i] == "電視遊樂器綜合討論"):
        #console_content.append(df2.tokens[i])
    #elif(df.category[i] == "智慧型手機"):
        #phone_content.append(df2.tokens[i])



In [18]:
#categories = ["電腦應用綜合討論", "電視遊樂器綜合討論", "智慧型手機"]
#all_contents = [computer_content, console_content, phone_content]

#data = zip(categories, all_contents)
#df3 = pd.DataFrame(list(data), columns=['category' ,'tokens'])

#df3.head(3)

In [19]:
#df2.to_csv('baha_dataset_freq.csv', sep='|', index=None)

In [20]:
#df2.head(2)